In [34]:
import pickle
import torch

In [35]:
folder="/data/home/umang/Trajectory_project/GPS-MTM/outputs/test_geolife/2025-08-29_11-10-58/test_outputs/random_masking_0.15_testing/ID"
pred_pickle_path="{}/predictions_batch_2.pkl".format(folder)
batch_pickle_path = "{}/ground_truth_batch_2.pkl".format(folder)
attention_pickle_path = "{}/attention_masks_batch_2.pkl".format(folder)
masks_pickle_path = "{}/masks_batch_2.pkl".format(folder)
with open(pred_pickle_path, 'rb') as f:
    predictions = pickle.load(f)

with open(batch_pickle_path, 'rb') as f:
    batch = pickle.load(f)

with open(attention_pickle_path, 'rb') as f:
    attention_masks = pickle.load(f)

with open(masks_pickle_path, 'rb') as f:
    masks = pickle.load(f)

In [36]:
masks['states'].shape

torch.Size([2549])

In [37]:
predictions["states"].shape

torch.Size([1, 2549, 198])

In [38]:
predictions["actions"].shape

torch.Size([1, 2549, 11])

In [39]:
# get the most frequently top-5 most repeated in ground_truth_list_States
# #don't use torch top-k..
ground_truth_list_states = []
for i in range(len(attention_masks)):
     # first zero mask is first time when attention_masks[i] becomes zero
    zero_indices = (attention_masks[i].flatten() == 0).nonzero(as_tuple=True)[0]
    first_zero_mask = zero_indices[0].item() if len(zero_indices) > 0 else attention_masks[i].numel()
    
    ground_truth_states= batch["states"][i, :first_zero_mask, :]
    ground_truth_states = torch.argmax(ground_truth_states, dim=-1)
    ground_truth_list_states.append(ground_truth_states)

k=4
ground_truth_list_states = torch.cat(ground_truth_list_states).flatten()
unique, counts = torch.unique(ground_truth_list_states, return_counts=True)
top_k_ground_truth = unique[torch.topk(counts, k=k).indices]
top_k_ground_truth

top_1_ground_truth = unique[torch.topk(counts, k=1).indices]
top_4_ground_truth = unique[torch.topk(counts, k=4).indices]    
# top_5_ground_truth = unique[torch.topk(counts, k=5).indices]
# top_10_ground_truth = unique[torch.topk(counts, k=10).indices]
# top_20_ground_truth = unique[torch.topk(counts, k=20).indices]


In [40]:
batch['actions'][0]

tensor([[68.0000,  0.0000,  6.0000,  ...,  0.2329,  0.1602, -0.1035],
        [68.0000,  1.0000,  0.0000,  ...,  0.2304,  0.1599, -0.1035],
        [68.0000,  1.0000,  0.0000,  ...,  0.2328,  0.1602, -0.1035],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [41]:
masks

{'states': tensor([1., 1., 1.,  ..., 0., 0., 0.], dtype=torch.float64),
 'actions': tensor([1., 1., 1.,  ..., 0., 0., 0.], dtype=torch.float64)}

In [42]:
predictions_list_states=[]
predictions_list_actions=[]
ground_truth_list_states=[]
ground_truth_list_actions=[]
correct_list=[]


# Get mask for this sequence
mask_states = masks["states"] == 1
mask_actions = masks["actions"] == 1

net_masks_states= attention_masks* masks["states"]
net_masks_actions= attention_masks* masks["actions"]

# Apply masks to get only positions where mask is 1
predictions_states = predictions["states"][net_masks_states.bool(), :]
predictions_actions = predictions["actions"][net_masks_actions.bool(), :]
ground_truth_states = batch["states"][net_masks_states.bool(), :]
ground_truth_actions = batch["actions"][net_masks_actions.bool(), :]

# take argmax of predictions states on last dimension

predictions_list_states.append(torch.argmax(predictions_states, dim=-1))
predictions_list_actions.append(torch.argmax(predictions_actions, dim=-1))
ground_truth_list_states.append(torch.argmax(ground_truth_states, dim=-1))
ground_truth_list_actions.append(torch.argmax(ground_truth_actions, dim=-1))

print(ground_truth_states)
correct = (predictions_states == ground_truth_states).float()
correct_list.append(correct)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [43]:
net_masks_states[..., 0:100]

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], dtype=torch.float64)

In [44]:
predictions_states[0:5, :]

tensor([[ -6.6315,  -2.2407,  -5.0313,  -6.2656,  -0.9350, -12.1295,  -5.5427,
           1.0801,  -6.7113,   8.5423,   5.2822,  -3.7694,  -3.2153,  -4.2131,
          -8.9773,   4.0336,  -9.8856,  -0.0458,  -4.3188,  -1.4296,   3.4820,
         -11.8377,  -9.4281,  -6.3491, -14.4032,  -9.1631,  -0.9988,  -5.5367,
           2.3195,  -1.5518,  -5.3880,  -8.8343,  -6.1586,  -8.7184,   1.7342,
          -6.4831,  -3.6416,  -4.3685,  -9.1126,  -7.8347, -11.8326,  -8.1309,
          -7.4324,  -4.4095,  -3.2334,  -5.2041,  -6.3862, -11.2189, -17.1146,
          -6.8851, -13.3963, -15.6818,  -7.6069,  -8.9787, -21.1135,  -8.8142,
          -4.2848, -10.4388,  -3.5807,  -7.8143,  -0.1464,  -9.4697,  -8.5382,
           0.9771,  -5.2748, -12.3274, -11.6152, -10.8967,  -8.8121,  -5.7624,
         -15.4035,  -4.2832,  -8.8877,  -1.5747, -12.2615, -16.0500, -13.2349,
          -9.2596,  -7.1876, -14.7222,  -0.4266, -12.3616,  -5.0494,  -8.6724,
          -9.5940,  -4.8098, -10.4608,  -3.8335, -24

In [45]:
predictions_states.shape,ground_truth_states.shape

(torch.Size([509, 198]), torch.Size([509, 198]))

In [46]:
predictions_list_states=torch.cat(predictions_list_states).flatten()
ground_truth_list_states=torch.cat(ground_truth_list_states).flatten()
predictions_list_states.shape, ground_truth_list_states.shape

(torch.Size([509]), torch.Size([509]))

In [47]:
predictions_list_states[0:200], ground_truth_list_states[0:200]

(tensor([  9,  10,   4,  48,   9,   9,  17,  17,   4,  17,  17,   9,   9,  34,
           4,   4,   4,  45,  45,   9,  60,   8,   9,  32,  59,  34,  34,  34,
         120, 120,  13,   9,   9,   4,   9,  15,   3,   3,  60,  13,  63,  15,
          15,  34,  13,   4,   9,  29,  59,   9,  25,  15,  15,  43,  43,  17,
          20,  26,  20, 106,  73,  13,   9,  10,  26,  17,  17,  17,  17,  15,
           3,   3,  20,  31,  55,  15,  82,  63,   9,   9,   4,   4,   4,   4,
           4,  73,  73,  15,  15,  25,  19,   4,   4,   4,   3,  15,  19,  29,
          20,  10,  25,  17,  17,  10,  25,  25,  19,   9,  12,   4,  39,   4,
           9,  39,  15,  15,   5,   4,   4,  18,   9,  15,  15,  15,  73,  44,
          39,  17,  17,  15,  15,   9,   9,   4,  26,  26,   9,   9,   6,   9,
           9,  13,  13,   9,  20,  15,   6,   3,   3,  17,  17,   9,   4,  13,
           1,   4,  15,  15,   4,   6,   6,  15,  78,  78,   4,  11,  15,  82,
           9,   4,  15,  55,  34,  82,  32,  86,   4

In [48]:
print(net_masks_states[..., 0:300])

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1.,

In [49]:
predictions_list_states[0:200]-ground_truth_list_states[0:200]

tensor([  0, -29,  -5,  39,   0,  -6, -12,   8,  -5, -19, -12,  -2,  -2,  15,
         -5, -13, -13,  28,  28, -48,  21,  -9, -35, -12,  15, -10, -10,  19,
         76,   0,   4,   0,  -8, -11,   0,   6,  -6,  -6,   7, -40,  46,   6,
         -5,  25, -13,  -6,  -6,  -9,  16, -48,  21, -29,  -2,   7,   7,   7,
         16,  17, -24,  89,  56,  -4,   0,   1, -10, -27,   2,   2,   2, -21,
        -16, -14, -16,  -5,  38,   6,  73,  27, -27, -35, -13, -32,  -9, -11,
          0,  30,  30,   6,  -2, -19,  10, -32, -40, -13, -12, -28,  10,   0,
         -9, -19, -14, -22,  -3,  -9,   8,  -4,   2,   5,  -5, -13,  30,  -5,
         -1,  30, -21,  -2, -39, -62, -40, -26, -20, -14, -21, -21,  64,  35,
          0, -22, -22, -24, -14,   5, -34, -25,  -3,  13, -34, -20, -37,  -8,
        -34,   4,   4,  -8,   3, -21, -30, -26, -26, -19,   2,   5, -25, -54,
        -28, -11,   6,  11, -13, -11, -11,  11,  75,  75,   1,   8,  12,  79,
        -11, -39, -29,  11,   0,  46,  23,  50,  -5, -16,   9, -

In [50]:
## Calculate accuracy USING predicted states and ground truth list states.
total_correct = 0
total_elements = 0
for pred_states, gt_states in zip(predictions_list_states, ground_truth_list_states):
    correct = (pred_states == gt_states).float()
    total_correct += correct.sum().item()
    total_elements += correct.numel()
accuracy = total_correct / total_elements if total_elements > 0 else 0
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.0589


In [51]:
# filter predictions and ground truth for each unique class
unique_classes = torch.unique(ground_truth_list_states)
filtered_predictions = {}
filtered_ground_truth = {}
for cls in unique_classes:
    mask = (ground_truth_list_states == cls)
    filtered_predictions[cls.item()] = predictions_list_states[mask]
    filtered_ground_truth[cls.item()] = ground_truth_list_states[mask]

# sort classes by frequency
class_counts = []
for cls in unique_classes:
    count = (ground_truth_list_states == cls).sum().item()
    class_counts.append((cls.item(), count))

# sort by count in descending order
class_counts.sort(key=lambda x: x[1], reverse=True)

# compute and print accuracy, precision, recall, and f1 score for each class sorted by frequency
print(f"{'Class':<5} {'Count':<6} {'Accuracy':<8} {'Precision':<9} {'Recall':<6} {'F1':<6}")
print("-" * 50)

for cls, count in class_counts:
    # Get predictions and ground truth for this class
    cls_predictions = filtered_predictions[cls]
    cls_ground_truth = filtered_ground_truth[cls]
    
    # Accuracy for this class
    accuracy = (cls_predictions == cls_ground_truth).float().mean().item()
    
    # For precision, recall, F1: need to consider this class vs all others
    # True positives: predicted this class and actually this class
    tp = ((predictions_list_states == cls) & (ground_truth_list_states == cls)).sum().item()
    
    # False positives: predicted this class but actually not this class
    fp = ((predictions_list_states == cls) & (ground_truth_list_states != cls)).sum().item()
    
    # False negatives: didn't predict this class but actually this class
    fn = ((predictions_list_states != cls) & (ground_truth_list_states == cls)).sum().item()
    
    # Calculate precision, recall, F1
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    print(f"{cls:<5} {count:<6} {accuracy:<8.4f} {precision:<9.4f} {recall:<6.4f} {f1:<6.4f}")

Class Count  Accuracy Precision Recall F1    
--------------------------------------------------
9     102    0.1275   0.1566    0.1275 0.1405
29    76     0.0263   0.1111    0.0263 0.0426
17    61     0.0656   0.1176    0.0656 0.0842
44    56     0.0179   0.2000    0.0179 0.0328
36    43     0.0000   0.0000    0.0000 0.0000
43    34     0.0294   0.2500    0.0294 0.0526
15    27     0.0000   0.0000    0.0000 0.0000
4     25     0.2400   0.0588    0.2400 0.0945
39    13     0.0769   0.1250    0.0769 0.0952
13    11     0.0000   0.0000    0.0000 0.0000
20    9      0.0000   0.0000    0.0000 0.0000
66    9      0.0000   0.0000    0.0000 0.0000
3     6      0.0000   0.0000    0.0000 0.0000
10    5      0.0000   0.0000    0.0000 0.0000
19    5      0.0000   0.0000    0.0000 0.0000
32    3      0.0000   0.0000    0.0000 0.0000
34    3      0.3333   0.0455    0.3333 0.0800
45    3      0.0000   0.0000    0.0000 0.0000
7     2      0.0000   0.0000    0.0000 0.0000
11    2      0.0000   0.0000 

In [52]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

# Convert to numpy for sklearn compatibility
gt_numpy = ground_truth_list_states.cpu().numpy()
pred_numpy = predictions_list_states.cpu().numpy()

# Overall metrics
overall_accuracy = accuracy_score(gt_numpy, pred_numpy)
macro_f1 = f1_score(gt_numpy, pred_numpy, average='macro')
weighted_f1 = f1_score(gt_numpy, pred_numpy, average='weighted')
macro_precision = precision_score(gt_numpy, pred_numpy, average='macro')
weighted_precision = precision_score(gt_numpy, pred_numpy, average='weighted')
macro_recall = recall_score(gt_numpy, pred_numpy, average='macro')
weighted_recall = recall_score(gt_numpy, pred_numpy, average='weighted')

print("Overall Performance Metrics:")
print(f"Overall Accuracy: {overall_accuracy:.4f}")
print(f"Macro F1 Score: {macro_f1:.4f}")
print(f"Weighted F1 Score: {weighted_f1:.4f}")
print(f"Macro Precision: {macro_precision:.4f}")
print(f"Weighted Precision: {weighted_precision:.4f}")
print(f"Macro Recall: {macro_recall:.4f}")
print(f"Weighted Recall: {weighted_recall:.4f}")

print("\nDetailed Classification Report:")
print(classification_report(gt_numpy, pred_numpy))

# Class-wise metrics accounting for imbalance
unique_classes = torch.unique(ground_truth_list_states)
class_counts = [(cls.item(), (ground_truth_list_states == cls).sum().item()) for cls in unique_classes]
class_counts.sort(key=lambda x: x[1], reverse=True)

print(f"\n{'Class':<5} {'Count':<6} {'Freq%':<6} {'Precision':<9} {'Recall':<6} {'F1':<6}")
print("-" * 50)

for cls, count in class_counts:
    freq_pct = (count / len(ground_truth_list_states)) * 100
    
    # Calculate metrics for this specific class
    cls_mask_gt = (gt_numpy == cls)
    cls_mask_pred = (pred_numpy == cls)
    
    tp = ((pred_numpy == cls) & (gt_numpy == cls)).sum()
    fp = ((pred_numpy == cls) & (gt_numpy != cls)).sum()
    fn = ((pred_numpy != cls) & (gt_numpy == cls)).sum()
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    print(f"{cls:<5} {count:<6} {freq_pct:<6.1f} {precision:<9.4f} {recall:<6.4f} {f1:<6.4f}")

Overall Performance Metrics:
Overall Accuracy: 0.0589
Macro F1 Score: 0.0197
Weighted F1 Score: 0.0603
Macro Precision: 0.0245
Weighted Precision: 0.1078
Macro Recall: 0.0336
Weighted Recall: 0.0589

Detailed Classification Report:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         6
           4       0.06      0.24      0.09        25
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         2
           8       0.00      0.00      0.00         0
           9       0.16      0.13      0.14       102
          10       0.00      0.00      0.00         5
          11       0.00      0.00      0.00         2
          12       0.00      0.00      0.00         2
          13       0.00      0.00      0.00        11
          15       0.00      0.00      0.00        27
          1

/data/home/umang/miniconda3/envs/mtm/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/data/home/umang/miniconda3/envs/mtm/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/data/home/umang/miniconda3/envs/mtm/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap

In [53]:
# get the number of correct predictions for classes in top_10_ground_truth
#top_k=[top_1_ground_truth, top_5_ground_truth, top_10_ground_truth, top_20_ground_truth]
top_k=[top_1_ground_truth, top_4_ground_truth]
for i in range(len(top_k)):
    filtered_gt = ground_truth_list_states[(ground_truth_list_states.unsqueeze(-1) == top_k[i]).any(dim=-1)]
    filtered_pred = predictions_list_states[(ground_truth_list_states.unsqueeze(-1) == top_k[i]).any(dim=-1)]

    # get the number of correct predictions and accuracy
    correct_predictions = (filtered_pred == filtered_gt).sum()
    accuracy = correct_predictions / filtered_gt.numel()
    print(f"Number of correct predictions in {top_k[i]} ground truth: {correct_predictions.item()}")
    print(f"Accuracy of correct predictions in {top_k[i]} ground truth: {accuracy.item()}")

Number of correct predictions in tensor([9]) ground truth: 13
Accuracy of correct predictions in tensor([9]) ground truth: 0.12745098769664764
Number of correct predictions in tensor([ 9, 29, 17, 44]) ground truth: 20
Accuracy of correct predictions in tensor([ 9, 29, 17, 44]) ground truth: 0.06779661029577255


### Proof that model is working properly below.

-0.0207 is the index for mask value to be filled in. Corresponding mask is shown below.

In [54]:
# (Pdb) x_sample[0,:,0]
# tensor([-0.9309, -2.4709, -0.9647, -2.5226, -0.9906, -2.6224, -1.0323, -2.6396,
#         -1.0287, -2.5965, -0.0207, -0.0207, -1.0252, -2.4983, -1.0098, -0.0207,
#         -1.0417, -2.3514, -0.0207, -0.0207, -0.0207, -2.3262, -0.9608, -2.4531,
#         -0.9609, -2.5430, -0.0207, -2.4288, -0.0207, -1.5484, -0.9251, -2.4177,
#         -0.0207, -0.0207, -0.8637, -2.3879, -0.8872, -0.0207, -0.9029, -2.3117,
#         -0.9281, -2.3534, -1.0126, -2.4598, -0.9165, -0.0207, -0.8610, -2.4906,
#         -0.0207, -2.5534, -1.0234, -0.8551, -2.4868, -0.8567, -2.4971, -0.9002,
#         -2.4162, -0.0207, -0.0207, -0.9232, -0.0207, -0.9814, -0.0207, -0.0207,
#         -2.3403, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207], device='cuda:0')

In [55]:
# (Pdb) masks['states']
# tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1,
#         1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
#         0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1,
#         1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0,
#         1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
#         0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
#         1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
#         1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
#         1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
#         1, 1, 0, 1, 1], device='cuda:0')